In [1]:
import re
from scipy.stats import pearsonr,spearmanr

In [2]:
level_dic = {"I级":1, "II级":2, "III级":3, "IV级":4}
def proc_level(cont1, cont2):
    if not pd.isna(cont1):
        # print(cont1)
        level =  re.search("I级|II级|III级|IV级",cont1).group(0)
    elif not pd.isna(cont2):
        # print(cont2)
        level = re.search("I级|II级|III级|IV级",cont2).group(0)
    else:
        level = "I级"
    return level_dic[level]

def fill_lvef(cont1, cont2):
    if pd.isna(cont2):
        return mean_mace if cont1 else mean_non_mace
    else:
        return cont2

In [3]:
import pandas as pd 
data = pd.read_excel('./data/data_gt_60.xlsx')
# data_lt_60 = data[data['年龄']>60]

In [5]:
# 2.全体TyG、TG/HDL与 冠脉病变支数、LVEF 、心功能分级 person/spearman相关性分析
# 前2是数值 分桶为4 跟后三分别做相关性分析
slct_col = ["TyG指数","TG/HDL","LM_LAD_LCX_RCA","LVEF","Killip分级","心功能分级","mace"]
slct_df = data[slct_col]

In [6]:
# 缺失值处理
slct_df.dropna(subset=['mace'])
mace_df = slct_df[slct_df['mace']==True]
non_mace_df = slct_df[slct_df['mace']==False]
# 对LVEF填充空值
mean_mace = int(mace_df.LVEF.mean())
mean_non_mace = int(non_mace_df.LVEF.mean())
slct_df["proc_LVEF"] = slct_df.apply(lambda x: fill_lvef(x.mace, x.LVEF),axis=1)

# 对Killip分级、心功能分级填充空值
slct_df["分级处理"] = slct_df.apply(lambda x: proc_level(x.Killip分级, x.心功能分级),axis=1)

slct_df = slct_df.dropna(subset=[
    "TyG指数","TG/HDL","LM_LAD_LCX_RCA","分级处理","mace","proc_LVEF"])[[
    "TyG指数","TG/HDL","LM_LAD_LCX_RCA","分级处理","mace","proc_LVEF"]]

Lesion_map = {"单支":1, "双支":2, "多支":3}
slct_df["LM_LAD_LCX_RCA"] = slct_df.apply(lambda x: Lesion_map[x.LM_LAD_LCX_RCA],axis=1)

/var/folders/6y/2k_kxtyd07v79br33f6gvjn00000gn/T/ipykernel_26642/2122809672.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slct_df["proc_LVEF"] = slct_df.apply(lambda x: fill_lvef(x.mace, x.LVEF),axis=1)
/var/folders/6y/2k_kxtyd07v79br33f6gvjn00000gn/T/ipykernel_26642/2122809672.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slct_df["分级处理"] = slct_df.apply(lambda x: proc_level(x.Killip分级, x.心功能分级),axis=1)


In [7]:
res_list = []

In [8]:
# "TyG指数","TG/HDL","LM_LAD_LCX_RCA","分级处理","mace","proc_LVEF"
pc = spearmanr(slct_df['TyG指数'],slct_df['LM_LAD_LCX_RCA'])
res_list.append(["TyG指数","LM_LAD_LCX_RCA","spearmanr", pc[0], pc[1]])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
pc = pearsonr(slct_df['TyG指数'],slct_df['LM_LAD_LCX_RCA'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","LM_LAD_LCX_RCA","pearsonr", pc[0], pc[1]])

# slct_df.corr(method='spearman')

spearmanr 相关系数0.06129936347006171, 显著性水平0.18939120502551163：
pearsonr 相关系数0.0821306426361706, 显著性水平0.07846085543870918：


In [9]:
pc = spearmanr(slct_df['TyG指数'],slct_df['分级处理'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","分级处理","spearmanr", pc[0], pc[1]])


pc = pearsonr(slct_df['TyG指数'],slct_df['分级处理'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","分级处理","pearsonr", pc[0], pc[1]])

# slct_df.corr(method='spearman')

spearmanr 相关系数0.1567181912276227, 显著性水平0.0007436743772008765：
pearsonr 相关系数0.15933167026689674, 显著性水平0.0006037552523844603：


In [10]:
pc = spearmanr(slct_df['TyG指数'],slct_df['mace'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","mace","spearmanr", pc[0], pc[1]])

pc = pearsonr(slct_df['TyG指数'],slct_df['mace'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","mace","pearsonr", pc[0], pc[1]])


# slct_df.corr(method='spearman')

spearmanr 相关系数0.31209140217781456, 显著性水平7.53278024352688e-12：
pearsonr 相关系数0.2863040513405108, 显著性水平3.968870216662081e-10：


In [11]:
pc = spearmanr(slct_df['TyG指数'],slct_df['proc_LVEF'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","proc_LVEF","spearmanr", pc[0], pc[1]])


pc = pearsonr(slct_df['TyG指数'],slct_df['proc_LVEF'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TyG指数","proc_LVEF","pearsonr", pc[0], pc[1]])


# slct_df.corr(method='spearman')

spearmanr 相关系数-0.06656600722867308, 显著性水平0.15404776763738695：
pearsonr 相关系数-0.11073043521871528, 显著性水平0.017513248931804778：


In [12]:
pc = spearmanr(slct_df['TG/HDL'],slct_df['LM_LAD_LCX_RCA'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))

res_list.append(["TG/HDL","LM_LAD_LCX_RCA","spearmanr", pc[0], pc[1]])


pc = pearsonr(slct_df['TG/HDL'],slct_df['LM_LAD_LCX_RCA'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","LM_LAD_LCX_RCA","pearsonr", pc[0], pc[1]])


# slct_df.corr(method='spearman')

spearmanr 相关系数0.047140165291966234, 显著性水平0.3130446599364736：
pearsonr 相关系数-0.0276511440424399, 显著性水平0.5541516171472499：


In [13]:
pc = spearmanr(slct_df['TG/HDL'],slct_df['分级处理'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","分级处理","spearmanr", pc[0], pc[1]])


pc = pearsonr(slct_df['TG/HDL'],slct_df['分级处理'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","分级处理","pearsonr", pc[0], pc[1]])

# slct_df.corr(method='spearman')

spearmanr 相关系数0.07772333657491044, 显著性水平0.09591999310358154：
pearsonr 相关系数0.013652629534960138, 显著性水平0.7702612398566288：


In [14]:
pc = spearmanr(slct_df['TG/HDL'],slct_df['mace'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","mace","spearmanr", pc[0], pc[1]])


pc = pearsonr(slct_df['TG/HDL'],slct_df['mace'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","mace","pearsonr", pc[0], pc[1]])

# slct_df.corr(method='spearman')

spearmanr 相关系数0.14719633868138637, 显著性水平0.0015472065362140811：
pearsonr 相关系数0.0033514443349583894, 显著性水平0.9428525123323455：


In [15]:
pc = spearmanr(slct_df['TG/HDL'],slct_df['proc_LVEF'])
print("spearmanr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","proc_LVEF","spearmanr", pc[0], pc[1]])


pc = pearsonr(slct_df['TG/HDL'],slct_df['proc_LVEF'])
print("pearsonr 相关系数{}, 显著性水平{}：".format(pc[0], pc[1]))
res_list.append(["TG/HDL","proc_LVEF","pearsonr", pc[0], pc[1]])


# slct_df.corr(method='spearman')

spearmanr 相关系数0.03605417650537891, 显著性水平0.4404563374662155：
pearsonr 相关系数-0.005519561775861166, 显著性水平0.9060199433328372：


In [40]:
col = "变量1 变量2 相关系数方法 相关系数 显著性水平".split(" ")
res_df = pd.DataFrame(res_list,
             columns=col)
res_df.to_excel('./data/相关性分析.xlsx',index=None)

In [ ]:
# 01均为连续变量

# Pearson相关，用于评估两个连续变量之间的线性关联强度
# 简单线性回归，也可以用来评估两个连续变量之间的相关性，但是需要区分自变量和因变量。
# 02含有有序分类变量

# Spearman相关，用于检验至少含有一个有序分类变量的关联强度和方向。
# Kendall's tau-b相关系数，用于检验至少有一个有序分类变量关联强度和方向的非参数分析方法。
# 03含有无序分类变量

# 卡方检验，常用于分析无序分类变量之间的相关性，也可以用于分析二分类变量之间的关系。但是该方法只能对是否具有相关性进行统计分析，不能反映关联强度。
# Fisher精确检验，可以用于检验任何R*C数据之间的相关关系。

In [36]:
"变量1 变量2 相关系数方法 相关系数 显著性水平".split(" ")

['变量1', '变量2', '相关系数方法', '相关系数', '显著性水平']

In [38]:
res_list

[['TyG指数',
  'LM_LAD_LCX_RCA',
  'spearmanr',
  0.05780963385673955,
  0.21588659672334268],
 ['TyG指数',
  'LM_LAD_LCX_RCA',
  'pearsonr',
  0.07824918383928745,
  0.09368672720827081],
 ['TyG指数', '分级处理', 'spearmanr', 0.05602446636580009, 0.23042541208564904],
 ['TyG指数', '分级处理', 'pearsonr', 0.0597234419005903, 0.2010444591208584],
 ['TyG指数', 'mace', 'spearmanr', 0.13316944292472438, 0.0042210882084548484],
 ['TyG指数', 'mace', 'pearsonr', 0.1306677970774441, 0.00500159830121159],
 ['TyG指数', 'proc_LVEF', 'spearmanr', -0.02678594237419428, 0.5666215340372875],
 ['TyG指数', 'proc_LVEF', 'pearsonr', -0.0718886700019655, 0.12364868029520167],
 ['TG/HDL',
  'LM_LAD_LCX_RCA',
  'spearmanr',
  0.05323883283846149,
  0.2544749444485749],
 ['TG/HDL',
  'LM_LAD_LCX_RCA',
  'pearsonr',
  -0.0004598745185323931,
  0.9921518353406531],
 ['TG/HDL', '分级处理', 'spearmanr', 0.0832345617825211, 0.07451778717706775],
 ['TG/HDL', '分级处理', 'pearsonr', 0.042779147031280565, 0.3599642286709478],
 ['TG/HDL', 'mace', '

In [ ]:
!pwd